In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
source('0_parameters.sh')


In [3]:
source(snpnet_misc_R)


In [4]:
covariates <- c('age', 'sex', paste0('PC', 1:10))
UKB_phes <- c('covid19_severe', 'covid19_test_result')


In [16]:
ukb_covid_phe %>%
str_replace('@@ukb_covid_version@@', ukb_covid_version) %>%
fread(
    colClasses = c('#FID'='character', 'IID'='character'),
    select = c('#FID', 'IID', 'population', 'split', UKB_phes, covariates)
) %>%
rename('FID'='#FID') %>%
filter(population == 'white_british') -> phe_WB_df


In [17]:
for(UKB_phe in UKB_phes){
    # add covariate-based score in the data frame
    phe_WB_df %>% left_join(
        phe_WB_df %>% 
        rename(!!'phe' := all_of(UKB_phe)) %>%
        compute_covar_score('phe', covariates, 'binomial') %>%
        rename(!!paste(UKB_phe, 'covar_score', sep='.') := 'covar_score'),
        by=c('FID', 'IID')
    ) -> phe_WB_df
}


In [18]:
colnames(phe_WB_df)

[1] "FID"                             "IID"                            
 [3] "population"                      "split"                          
 [5] "covid19_severe"                  "covid19_test_result"            
 [7] "age"                             "sex"                            
 [9] "PC1"                             "PC2"                            
[11] "PC3"                             "PC4"                            
[13] "PC5"                             "PC6"                            
[15] "PC7"                             "PC8"                            
[17] "PC9"                             "PC10"                           
[19] "covid19_severe.covar_score"      "covid19_test_result.covar_score"

In [19]:
phe_WB_df %>% count(population, split)

population,split,n
<chr>,<chr>,<int>
white_british,test,67425
white_british,train,235991
white_british,val,33713


In [21]:
phe_WB_df %>%
rename('#FID' = 'FID') %>%
fwrite(file.path(data_d, 'UKB_PRS_eval', 'UKB.WB.phe'), sep='\t', na = "NA", quote=F)
